In [ ]:
import os
from dotenv import load_dotenv
import openai
# Load environment variables from .env file
load_dotenv()

True

In [ ]:
import os

from autogen import ConversableAgent
from autogen.agentchat.contrib import llamaindex_conversable_agent
llamaindex_conversable_agent()

In [45]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [ ]:
result = joe.initiate_chat(cathy, message="Cathy, praise Mrigha.", max_turns=2)
print(result.chat_history)

joe (to cathy):

Cathy, praise Mrigha.

--------------------------------------------------------------------------------
cathy (to joe):

Oh sure Joe, happy to oblige. Folks, can we all give a round of applause for our friend here, Mrigha? Good to have you here, Mrigha. You know, I heard Mrigha is so good at organizing, even their sock drawer has a seating chart!

--------------------------------------------------------------------------------
joe (to cathy):

That's true, Cathy, and don't forget about their spice rack! It's alphabetized... in Latin! You know you're a true organizer when you've got your paprika next to your piper nigrum!

--------------------------------------------------------------------------------
cathy (to joe):

Oh, absolutely, Joe! That's what I call taking 'seasoning' to a whole new level! I bet when Mrigha says 'Pass the salt', you have to ask 'In which language?' They're on another league; I still can't find my phone half of the time!

-----------------------

: 

Testing termination of chat
https://microsoft.github.io/autogen/0.2/docs/tutorial/chat-termination

In [47]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here goes:

Why don't scientists trust atoms?

Because they make up everything!

GOOD BYE!

--------------------------------------------------------------------------------


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's one. Why don't scientists trust atoms? 

Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

That's a good one, Cathy! Here's mine - Why don't some fish play piano?

Because you can't tuna fish!

--------------------------------------------------------------------------------
cathy (to joe):

Oh, that's a good one, Joe! I definitely didn't sea that punchline coming. It's got me hooked!

--------------------------------------------------------------------------------
[{'content': 'Cathy, tell me a joke.', 'role': 'assistant', 'name': 'joe'}, {'content': "Sure, here's one. Why don't scientists trust atoms? \n\nBecause they make up everything!", 'role': 'user', 'name': 'cathy'}, {'content': "That's a good one, Cathy! Here's mine - Why don't some fish play piano

In [57]:
import os
from autogen import  AssistantAgent
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, GPTVectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.retrievers import VectorIndexRetriever
# Load OpenAI API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
# Load and process documents (ensure you have a `documents/` folder)
documents = SimpleDirectoryReader(input_files = [r"D:\docs\LLM_RAG_structureddata\autogen\samplefile.pdf"]).load_data()

# Create a vector index
index = GPTVectorStoreIndex.from_documents(documents)
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)
# Create a query engine for retrieval



In [ ]:
index.as_retriever()

In [62]:
from llama_index.core.agent import AgentRunner
from llama_index.llms import openai
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

llm =  openai.OpenAI()
index = GPTVectorStoreIndex.from_documents(documents)
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)
retrieval_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(name="document_retriever", description="Retrieves relevant document sections"),
)


agent = AgentRunner.from_llm([retrieval_tool], llm=llm, verbose=True)

In [21]:
query_engine = index.as_query_engine()
llama_agent = OpenAIAgent.from_tools(
    tool_retriever=retriever,
    verbose=True
)

In [ ]:
# Define LlamaIndexConversableAgent with RAG capabilities
from autogen import ConversableAgent, UserProxyAgent

llama_index_agent = LLamaIndexConversableAgent(
    name="LlamaIndexAgent",
    description='llamaindex agent to query information from the vector store',
    llama_index_agent=agent, # Uses LlamaIndex for retrieval
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

# Define an AI assistant for responses
assistant = AssistantAgent(
    name="AI_Assistant",
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input,
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower() or None
)

In [ ]:
human_proxy.initiate_chat(llama_index_agent, message='what is the pdf about?')

human_proxy (to LlamaIndexAgent):

what is the pdf about?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Added user message to memory: what is the pdf about?
=== Calling Function ===
Calling function: document_retriever with args: {"input":"pdf"}
Got output: The information provided pertains to a tax invoice generated in PDF format.

LlamaIndexAgent (to human_proxy):

The PDF document is a tax invoice. If you need more specific information or details from the document, please let me know.

--------------------------------------------------------------------------------
human_proxy (to LlamaIndexAgent):

can you give me little more details?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Added user message to memory: can you give me little more details?
=== Calling Function ===
Calling function: document_retriever with args: {"input":"tax invoice"}
Got output:

In [ ]:
llama_index_agent._llama_index_agent


autogen.agentchat.contrib.llamaindex_conversable_agent.LLamaIndexConversableAgent

In [ ]:
user_proxy.initiate_chat(llama_index_agent, message='what is this pdf about?')

LlamaIndexAgent (to user_proxy):

what is this pdf about?

--------------------------------------------------------------------------------
user_proxy (to LlamaIndexAgent):

what is this pdf about?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


AttributeError: 'LLamaIndexConversableAgent' object has no attribute 'chat'